In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


# определим кодировки
#from chardet.universaldetector import UniversalDetector # Импортируем субмодуль chardet.universaldetector
#detector = UniversalDetector()
#with open('data/Справочник Номенклатуры.xlsx', 'rb') as fh:
#    for line in fh:
#        detector.feed(line)
#        if detector.done:
#            break
#detector.close()

nomen_data = pd.read_excel('data/Справочник Номенклатуры.xlsx')
sales_data = pd.read_excel('data/Продажи_дляП1.xlsx')
price_data = pd.read_excel('data/ПрайсДляП.xlsx')
stock_data = pd.read_excel('data/Остатки_дляП1.xlsx')

In [2]:
display(nomen_data.shape)
m = nomen_data.shape[1] 
nomen_data = nomen_data.dropna( how='any' ,thresh=m-2 ,axis=0)



(47348, 12)

In [3]:

#Обработаем остатки
stock_data['Дата'] = pd.to_datetime(stock_data['ДАТА'], dayfirst=True) 
stock_data['Артикул'] = stock_data['АРТИКУЛ']
stock_data = stock_data.drop(['ДАТА','Маркер', 'АРТИКУЛ'],axis=1)

In [4]:
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], dayfirst=True)
sales_data['Дата'] = sales_data['Дата'].dt.date
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], dayfirst=True)
sales_data['Артикул'] = sales_data['Артикул']

In [5]:
display(nomen_data.head())
display(sales_data.head())
display(price_data.head())
display(stock_data.head())

,Артикул,Номенклатура,Вес,Объем,Коэффициент,Номенклатурная группа,Ассортиментная группа,Проект,Категория,Основной поставщик,Артикул производителя,Статус
0,182260/п,"Бокал д/вина, 0.2л, кристалл, ПС",9.00,0.060,79.0,Бокалы,Бокалы,1. Одноразовая посуда,X5 Retail Group,АССОРТИ-ТВЕРЬ ООО,NaN,Действующая
1,182261,"Бокал д/вина, 0.20л, кристалл, ПС",1.20,0.021,12.0,Бокалы,Бокалы,1. Одноразовая посуда,X5 Retail Group,АССОРТИ-ТВЕРЬ ООО,182261,Действующая
2,182272//,"Бокал д/мартини, 0.1л, прозрачн., кристалл, ПС",1.96,0.064,16.0,Бокалы,Бокалы,1. Одноразовая посуда,АШАН,ВЗЛП ООО,1008,Действующая
3,182271/в,"Бокал д/вина, 0.2л, прозрачн., кристалл, ПС",3.50,0.025,10.0,Бокалы,Бокалы,1. Одноразовая посуда,Маркетплейс Wildberries,ВЗЛП ООО,1011,Действующая
4,182272/в,"Бокал д/мартини, 0.1л, прозрачн., кристалл, ПС",3.20,0.075,32.0,Бокалы,Бокалы,1. Одноразовая посуда,Маркетплейс Wildberries,ВЗЛП ООО,1008,Действующая


,Артикул,КЛИЕНТ,Менеджер,Дата,Продажи бе
0,440038,ВОСТОК-ЗАПАД ООО,Поспелова Елена Олеговна,2022-06-06,4900.0
1,440038а,ВОСТОК-ЗАПАД ООО,Поспелова Елена Олеговна,2022-06-06,4000.0
2,153501ч,ИНТЕРНЕТ РЕШЕНИЯ ООО,Кузнецова Татьяна Анатольевна,2022-06-06,2.0
3,159501,ИНТЕРНЕТ РЕШЕНИЯ ООО,Кузнецова Татьяна Анатольевна,2022-06-06,2.0
4,153501ч,ИНТЕРНЕТ РЕШЕНИЯ ООО,Кузнецова Татьяна Анатольевна,2022-06-06,2.0


,Артикул,Цена
0,461620гр,42.00
1,481610,171.85
2,477083б,965.19
3,47630200,207.05
4,268348а/,121.48


,Остаток бе,Дата,Артикул
0,626.4,2022-06-06,100130
1,188.0,2022-06-06,100160
2,31.5,2022-06-06,103301
3,28.0,2022-06-06,103321
4,7.0,2022-06-06,103331


In [6]:
# сцепим вместе все данные в одну таблицу
myst_data = pd.merge(left = stock_data, right= sales_data, on = ['Дата','Артикул'], how = 'outer')
myst_data = pd.merge(left = myst_data, right= price_data, on = 'Артикул', how = 'left')
myst_data = pd.merge(left = myst_data, right= nomen_data, on = 'Артикул', how = 'left')
myst_data = myst_data.fillna({'Продажи бе':0, 'Остаток бе':0}) # уберем отсутствующие продажи и остатки
myst_data['Продажи руб'] = myst_data['Продажи бе'] * myst_data['Цена']
myst_data['Остаток руб'] = myst_data['Остаток бе'] * myst_data['Цена']
myst_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250492 entries, 0 to 250491
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Остаток бе             250492 non-null  float64       
 1   Дата                   250492 non-null  datetime64[ns]
 2   Артикул                250492 non-null  object        
 3   КЛИЕНТ                 73388 non-null   object        
 4   Менеджер               73388 non-null   object        
 5   Продажи бе             250492 non-null  float64       
 6   Цена                   248063 non-null  float64       
 7   Номенклатура           250154 non-null  object        
 8   Вес                    250149 non-null  float64       
 9   Объем                  250151 non-null  float64       
 10  Коэффициент            250154 non-null  float64       
 11  Номенклатурная группа  250154 non-null  object        
 12  Ассортиментная группа  249890 non-null  obje

In [7]:
# добавим дополнительные показатели
# Добавим данные о клиентах к артикулу (доля лучшего/лучший клиент/сколько всего клиентов)
# создадим базовые данные для сборки информации по артикулам
art_data = myst_data.groupby(
                                by=['Артикул'],
                                as_index=False
                                )['Продажи руб'].sum()

#создадим дата сет артикул-клиент
client_data = myst_data.groupby(
                                by=['Артикул', 'КЛИЕНТ'],
                                as_index=False
                                )['Продажи руб'].sum().sort_values(by='Продажи руб', ascending=False)

# функция определения имени клиента с макс продажами
def get_max_client_name(art):
    if len(client_data[client_data['Артикул'] == art]['КЛИЕНТ'].values)<1:
        return np.nan
    else: 
        return client_data[client_data['Артикул'] == art]['КЛИЕНТ'].iloc[0]

#создадим поля Продажи макс клиента, Макс клиент и Доля макс клиента
art_data['Продажи макс клиента руб'] = art_data['Артикул'].apply(lambda x: client_data[client_data['Артикул'] == x]['Продажи руб'].max())
art_data['Макс Клиент'] = art_data['Артикул'].apply(get_max_client_name)
art_data['Доля Макс Клиента'] = art_data['Продажи макс клиента руб'] / art_data['Продажи руб']

#создадим поле Число клиентоа
client_data = client_data['Артикул'].value_counts()
def get_client_count(art):
    try:
        res = client_data.loc[art]
    except:
        return np.nan
    return client_data.loc[art]

art_data['Число клиентов'] = art_data['Артикул'].apply(get_client_count) 

display(art_data.head())

,Артикул,Продажи руб,Продажи макс клиента руб,Макс Клиент,Доля Макс Клиента,Число клиентов
0,0,0.00,NaN,NaN,NaN,NaN
1,100120,7020.00,2340.00,ОАНО Школа НИКА,0.333333,3.0
2,100130,116093.25,41079.15,Селедцова О.К.,0.353846,14.0
3,100160,104849.64,31298.40,Садыков Рамиз Нюсратович ИП,0.298507,14.0
4,103301,2530.85,2530.85,СТОРЕН ООО,1.000000,1.0


In [8]:
# Создадим поле связанное с частотой продаж
freq_data = myst_data.groupby(
                                by=['Артикул','Дата'],
                                as_index=False
                                )['Продажи руб'].sum()
display(freq_data.head())
date_num = myst_data['Дата'].unique().shape[0]

def get_freq_count(art):
    return freq_data[(freq_data['Артикул'] == art) & (freq_data['Продажи руб'] > 0)].shape[0]/date_num

art_data['Частота'] = art_data['Артикул'].apply(get_freq_count) 

# cколько всего активных дат в таблице


,Артикул,Дата,Продажи руб
0,0,2022-06-06,0.0
1,0,2022-06-07,0.0
2,0,2022-06-08,0.0
3,0,2022-06-09,0.0
4,0,2022-06-10,0.0


In [9]:
#Создадим статистические метрики по датам
freq_data_mod = freq_data.groupby(
                                by=['Артикул'],
                                as_index=False
                                )['Продажи руб'].agg(['mean','std','median'])

display(freq_data_mod.head())
art_data['Среднеедневная'] = art_data['Артикул'].apply(lambda x: freq_data_mod['mean'].loc[x]) 
art_data['Медиана'] = art_data['Артикул'].apply(lambda x: freq_data_mod['median'].loc[x]) 
art_data['std'] = art_data['Артикул'].apply(lambda x: freq_data_mod['std'].loc[x]) 
art_data['std'] = art_data['std'] / art_data['Среднеедневная']
#art_data['std'] = art_data['std'].apply(lambda x: x if x<3 else 3)
display(art_data.head())

,mean,std,median
Артикул,,,
0,0.000000,0.000000,0.0
100120,270.000000,762.401469,0.0
100130,1612.406250,5062.016164,0.0
100160,1456.245000,2695.544847,0.0
103301,35.150694,298.263533,0.0


,Артикул,Продажи руб,Продажи макс клиента руб,Макс Клиент,Доля Макс Клиента,Число клиентов,Частота,Среднеедневная,Медиана,std
0,0,0.00,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,NaN
1,100120,7020.00,2340.00,ОАНО Школа НИКА,0.333333,3.0,0.034091,270.000000,0.0,2.823709
2,100130,116093.25,41079.15,Селедцова О.К.,0.353846,14.0,0.136364,1612.406250,0.0,3.139417
3,100160,104849.64,31298.40,Садыков Рамиз Нюсратович ИП,0.298507,14.0,0.284091,1456.245000,0.0,1.851024
4,103301,2530.85,2530.85,СТОРЕН ООО,1.000000,1.0,0.011364,35.150694,0.0,8.485281


In [10]:
#создадим поле связанное с наличием
stock_data = myst_data.groupby(
                                by=['Артикул','Дата'],
                                as_index=False
                                )['Остаток руб'].mean()
display(stock_data.head())

def get_stock_count(art):
    return stock_data[(stock_data['Артикул'] == art) & (stock_data['Остаток руб'] > 0) ].shape[0]/date_num

art_data['Наличие'] = art_data['Артикул'].apply(get_stock_count)
display(art_data.shape)

art_data = pd.merge(left = art_data, right= nomen_data, on = 'Артикул', how = 'left')
display(art_data.shape)

,Артикул,Дата,Остаток руб
0,0,2022-06-06,NaN
1,0,2022-06-07,NaN
2,0,2022-06-08,NaN
3,0,2022-06-09,NaN
4,0,2022-06-10,NaN


(4159, 11)

(4159, 22)

In [11]:
# Создадим наши категориальные признаки медианы
art_data['Категория медианы'] = art_data['Медиана']/art_data['Среднеедневная']

def set_median_cat(cat):
    if cat == np.nan:
        return np.nan
    elif cat >= 1.0:
        return 'Разовый по медиане'
    elif cat == 0:
        return 'Редкий по медиане'
    else:
        return 'Стандарт по медиане'

art_data['Категория медианы'] = art_data['Категория медианы'].apply(set_median_cat)

# Создадим наши категориальный признак Продажи/Средние продажи
art_data['Наклон нормальности'] =  art_data['Продажи руб']/art_data['Среднеедневная']
k_norm = art_data[art_data["Доля Макс Клиента"]<0.75]['Наклон нормальности'].mean()
# если Продажи руб - (Среднее - 2000)* коэф >= 0
art_data['Категория нормальности'] = art_data['Продажи руб'] -  (art_data['Среднеедневная']) * k_norm
art_data['Категория нормальности'] = art_data['Категория нормальности'].apply(lambda x: 'Нормальная' if x>=0 else 'Ненормальная')

art_data['Категория Стат'] = art_data['Категория нормальности'] + ' ' + art_data['Категория медианы']

In [12]:
#Создадим логарифмированные признаки и разделим на Стат Категории Способом 2
art_data['лог медиана'] = art_data['Медиана'].apply(np.log)
art_data['лог продажи'] = art_data['Продажи руб'].apply(np.log)
art_data['лог среднее'] = art_data['Среднеедневная'].apply(np.log)
art_data['k прод к мед'] = art_data['Продажи руб'] / art_data['Медиана']

def set_cat_var2(art):
    if art_data[art_data['Артикул'] == art]['Медиана'].sum() == 0:
        return 'Редкая по медиане'
    elif np.abs(art_data[art_data['Артикул'] == art]['k прод к мед'].sum()  - 1) < 0.1:
        return 'Разовая продажа'
    elif np.abs(art_data[art_data['Артикул'] == art]['k прод к мед'].sum()  - 2) < 0.1:
        return '2х разовая продажа'
    elif art_data[art_data['Артикул'] == art]['Наличие'].median() < 0.3:
        return 'Разряженные'
    else:
        return 'Хорошие'
    
art_data['Категория Стат 2'] = art_data['Артикул'].apply(set_cat_var2)


In [23]:
import plotly.graph_objs as go 
df = art_data[(art_data['Доля Макс Клиента'] <=1 ) & (art_data['Доля Макс Клиента'] > 0 )].copy()
#df['Продажи руб'] = df['Продажи руб'].apply(lambda x: x if x< 500000 else 500000)
df['Продажи руб'] = df['Продажи руб'].apply(lambda x: 1 if x< 1 else x)

#fig = px.histogram(df, x= 'Продажи руб', 
#                 title='Распределение продаж', width=1200, height=500,
#                 marginal="box",text_auto=True)
#fig.show()

#fig = px.histogram(df, x= 'Число клиентов', 
#                 title='Распределение числа клиентов', width=1200, height=500,
#                 marginal="box",text_auto=True)
#fig.show()
#
#fig = px.histogram(df, x= 'Частота', 
#                 title='Распределение частоты заказов', width=1200, height=500,
#                 marginal="box",text_auto=True)
#fig.show()
#
#fig = px.histogram(df, x= 'Наличие', 
#                 title='Распределение наличия', width=1200, height=500,
#                 marginal="box",text_auto=True)
#fig.show()
#
#fig = px.histogram(df, x= 'Доля Макс Клиента', 
#                 title='Распределение доли макс клиента', width=1200, height=500,
#                 marginal="box",text_auto=True)
#fig.show()
#
#fig = px.scatter_matrix(df,
#    dimensions=["Продажи руб", "Доля Макс Клиента", "Число клиентов", "Частота", 'Наличие','Медиана','std','Среднеедневная'], width=1200, height=1200, title='Вычисление зависимостей в номенклатуре')
#fig.show()


#fig = px.scatter(df, x= 'Число клиентов', y= 'Частота', color='Продажи руб', color_continuous_scale='oryel',
#                 title='Зависимость числа клиентов и частоты покупок', width=1000, height=700)
#                 #marginal_x="histogram", marginal_y="histogram")
#fig.show()

#fig = px.scatter(df, x= 'Среднеедневная', y= 'Продажи руб', log_x=True, log_y=True,
#                 title='Зависимость среднедневных продаж и суммы продаж', width=1000, height=700,
#                  color = 'Категория Стат')


#x = np.arange(0, 60000, 1)
#fig.add_trace(go.Scatter(x=x, y=k_norm*(x)  , name='Линия Продажа = коэф * ср день')) # Рисуем y = x

#fig.show()

df2 = df
df2['Медиана1'] = df['Медиана'].apply(lambda x: x+1)
#df2['Число клиентов'] = df2['Число клиентов'].apply(lambda x: 50 if x>50 else x)

#fig = px.scatter(df2, x= 'Среднеедневная', y= 'Медиана1', log_x=True, log_y=True, 
#                 title='Зависимость средних и медианных продаж', width=1000, height=700,
#                  color = 'Категория Стат')
#fig.show()

def set_cat3(art):
    if (df2[df2['Артикул'] == art]['Число клиентов'].sum() < 2) and (df2[df2['Артикул'] == art]['Продажи руб'].sum() < 5000):
        return 'Кат1'
    elif (df2[df2['Артикул'] == art]['Число клиентов'].sum() < 5) and (df2[df2['Артикул'] == art]['Продажи руб'].sum() < 10000):
        return 'Кат2'
    else :
        return 'Кат3'
    
    
df2['Cat3'] = df2['Артикул'].apply(set_cat3)

fig = px.scatter(df2, x= 'Продажи руб', y= 'Число клиентов', color = 'Cat3' ,
                 width=1000, height=700)
fig.show()

df_cat = df.groupby(
                                by='Cat3',
                                as_index=False
                                )['Cat3'].value_counts()

fig = px.bar(df_cat, y="count", x='Cat3',text_auto=True)
fig.show()






fig = px.scatter(df2, x= 'Продажи руб', y= 'Медиана1', log_x=True, log_y=True, 
                 title='Зависимость средних и медианных продаж', width=1000, height=700,
                  color = 'Категория Стат 2')
fig.show()

#fig = px.scatter(df2, x= 'Продажи руб', y= 'Медиана1', log_x=True, log_y=True, 
#                 title='Зависимость средних и медианных продаж', width=1000, height=700,
#                  color = 'Наличие')
#fig.show()
#
#fig = px.scatter(df2, x= 'Продажи руб', y= 'Медиана1', log_x=True, log_y=True, 
#                 title='Зависимость средних и медианных продаж', width=1000, height=700,
#                  color = 'std')
#fig.show()




fig = px.scatter_3d(
    data_frame=df2, #DataFrame
    x = 'Продажи руб', #ось абсцисс
    y = 'Медиана1', #ось ординат
    z = 'Среднеедневная', #ось аппликат
    color="Категория Стат 2", #расцветка в зависимости от страны
    log_x=True, 
    log_y=True,
    log_z=True,
    width=1000,
    height=700
)

fig.show()




#display(df[df["Число клиентов"] > 100])
#display(k_norm)

#df1 = df[df['Категория Стат'] == 'Нормальная Стандарт по медиане']
#fig = px.scatter(df1, y= "Продажи руб", x= "Число клиентов", 
 #                title='Зависимость средних и медианных продаж', width=1000, height=700,
 #                 color = 'Доля Макс Клиента')
#fig.show()




df_cat = df.groupby(
                                by='Категория Стат 2',
                                as_index=False
                                )['Продажи руб'].sum()

fig = px.bar(df_cat, y="Продажи руб", x='Категория Стат 2', text_auto=True)
fig.show()

df_cat = df.groupby(
                                by='Категория Стат 2',
                                as_index=False
                                )['Категория Стат 2'].value_counts()

fig = px.bar(df_cat, y="count", x='Категория Стат 2',text_auto=True)
fig.show()

df.sort_values(by='Продажи руб', ascending=False).to_excel('data/analysis1.xlsx')

### Категоризация 1.

Товары в доль линии концентрации Продажи руб / Среденедневные. Обозначим как "Пропорция продаваемости" - Правильная/Неправильная

### Категоризация 2.

Соотношение медианы и среднедневных продаж
* Медиана = 0. (Редкие по медиане)
* Медиана = Среднее (Разовые по медиане)
* Медиана != Среднее (Стандартные по медиане)

### Категоризация 3Д. Альтернатива. 

Из анализа логарифмированной 3д диаграммы рассеяния по параметрам: среднее, продажи, медиана видно.
* Медиана = 0. 
Правильная категория. Но в ней точно нужна дополнительное деление и дополнительный анализ, чтобы проверить, что разделение на Нормальные и ненормальные разумное.
В этой группе в целом видно, что есть артикулы с разными значениями по std, зависимости от максб числу клиентов и т.д.
* Медиана = Среднее. Правильная категоризация. Выделяется четко линия выбросов.
При медиане = 1 выделась еще одна зона тут. Это товары очень близкие у условно разовым, зависимость от одного клиента у них высокая, отгрузки резкие, но хорошая низкая вариация.
* Стандартные по медиане же деляться на два кучкующихся облака.
Одно облако ближе к зоне "нормальных товаров по медиане и с нормальным соотношением Продаж и среднего дня". Второе облако ближе к зоне "Условно разовых". И эти же товары сущесвенно разные по всем показателям типа std, частота, число клиентов.

#### Резюме:
* Кат 1.0 Редкие по медиане. Точно разделиться на какие-то еще группы. Доанализировать.
* Кат 2.0 Разовые по медиане. Возможно разделиться на стабильные и нестабильные по STD. Проверить.
* Кат 3.0 Стандартные по медиане. Разделены на нормальные и нет. Возможно нужно другое деление т.к. ненормальные частично тяготеют к нормальным, а частично к Разовым по медиане и облака выделены. Там суммарно мало артикулов, можно что-то придумать.

In [14]:
# Проведем анализ группы артикулов Медиана = 0
df = art_data[(art_data['Доля Макс Клиента'] <=1 ) & (art_data['Доля Макс Клиента'] > 0 )].copy()
df = df[df['std'] >= 0]
#df = df[df['Доля Макс Клиента'] > 1]
df = df[df['Категория Стат 2'] == 'Редкая по медиане']
df['Продажи руб'] = df['Продажи руб'].apply(lambda x: 1 if x< 1 else x)
#"Продажи руб", "Доля Макс Клиента", "Число клиентов", "Частота", 'Наличие','std','Среднеедневная'

#fig = px.scatter_matrix(df,
 #   dimensions=["Продажи руб", "Доля Макс Клиента", "Число клиентов", "Частота", 'Наличие','std','Среднеедневная'], width=1200, height=1200, title='Вычисление зависимостей в номенклатуре')
#fig.show()

fig = px.scatter_3d(
    data_frame=df, #DataFrame
    x = 'Продажи руб', #ось абсцисс
    y = 'Наличие', #ось ординат
    z = 'Доля Макс Клиента', #ось аппликат
    color="Категория Стат", #расцветка в зависимости от страны
    log_x=True, 
    #log_y=True,
    #log_z=True,
    width=1000,
    height=700
)

fig.show()

Эта выборка сложнее для анализа.
Предлагаю разделить так:
* Выделить группу зависимых от 1го клиента - явно вырисовывается плоскость. - это 2
* Выделить группу нормальных по Стат Критериям первого варианта. По сути - это товары с высокой долей наличия - это 2
* Выделить группу наличие менее 30% по аналогии с делением товаров с незанулящейся медианой - это 2

Итого будем иметь 8 шкал. Попробуем эти шкалы проанализировать.
Вообще по текущим картинкам товары с медианой ноль - это черный ящик неуправляемого ассортимента. Либо нам не хватает критериально важных данных.

In [15]:
# Выделить 8 критериальных данных и посмотрим на них
df['Один клиент'] = df['Доля Макс Клиента'].apply(lambda x: '1 клиент' if x == 1 else 'N клиентов')

def set_ex_stat(art):
    if df[df['Артикул'] == art]['Наличие'].sum() >= 0.7:
        return 'Частое наличие'
    elif df[df['Артикул'] == art]['Наличие'].sum() > 0.3:
        return 'Среднее наличие'
    else:
        return 'Редкое наличие'
df['Наличие 2'] = df['Артикул'].apply(set_ex_stat)

df['Категория редкопродаваемых'] = df['Один клиент'] + ' ' + df['Наличие 2']

df['Финальная категория'] = 

SyntaxError: invalid syntax (1912414427.py, line 15)

In [ ]:
fig = px.scatter_3d(
    data_frame=df, #DataFrame
    x = 'Продажи руб', #ось абсцисс
    y = 'Наличие', #ось ординат
    z = 'Доля Макс Клиента', #ось аппликат
    color='Категория редкопродаваемых', #расцветка в зависимости от страны
    log_x=True, 
    #log_y=True,
    #log_z=True,
    width=1000,
    height=700
)

fig.show()

df_cat = df.groupby(
                                by='Категория редкопродаваемых',
                                as_index=False
                                )['Продажи руб'].sum()

fig = px.bar(df_cat, y="Продажи руб", x='Категория редкопродаваемых', text_auto=True)
fig.show()

df_cat = df.groupby(
                                by='Категория редкопродаваемых',
                                as_index=False
                                )['Категория редкопродаваемых'].value_counts()

fig = px.bar(df_cat, y="count", x='Категория редкопродаваемых',text_auto=True)
fig.show()

In [ ]:
# Дальше посмотрим на группу N клиентоа Частое наличие
#"Продажи руб", "Доля Макс Клиента", "Число клиентов", "Частота", 'Наличие','std','Среднеедневная'
df['Продажи руб'] = df['Продажи руб'].apply(lambda x: 100000 if x>100000 else x)

Osales = df['Среднеедневная'].max()
Ocount = df['Число клиентов'].max()
Ofreq = df['Частота'].max()

df['Частота норм'] = df['Частота']/Ofreq - 1
df['Число клиентов норм'] = df['Число клиентов']/Ocount - 1
df['Среднее норм'] = df['Среднеедневная']/Osales - 1
df['Rad'] = (df['Частота норм']**2 + df['Число клиентов норм']**2 + df['Среднее норм']**2)*(1/2)
df['Rad cat'] = df['Rad'].apply(lambda x: 'Good' if x<1.2 else 'Bad')



print(Osales, Ocount, Ofreq)

df = df[df['Категория редкопродаваемых'] == 'N клиентов Частое наличие']
fig = px.scatter_3d(
    data_frame=df, #DataFrame
    x = 'Число клиентов', #ось абсцисс
    y = 'Среднеедневная', #ось ординат
    z = 'Частота', #ось аппликат
    color='Rad cat', #расцветка в зависимости от страны
    #log_x=True, 
    #log_y=True,
    #log_z=True,
    width=1000,
    height=700
)

fig.show()


df_cat = df.groupby(
                                by='Rad cat',
                                as_index=False
                                )['Продажи руб'].sum()

fig = px.bar(df_cat, y="Продажи руб", x='Rad cat', text_auto=True)
fig.show()

df_cat = df.groupby(
                                by='Rad cat',
                                as_index=False
                                )['Rad cat'].value_counts()

fig = px.bar(df_cat, y="count", x='Rad cat',text_auto=True)
fig.show()

25629.854791666665 40.0 0.4090909090909091


In [ ]:
df = myst_data[myst_data['Артикул'] == '349054/']
df_sales = df.groupby(
                                by=['Дата'],
                                as_index=False
                                )['Продажи руб'].sum()
df_stock = df.groupby(
                                by=['Дата'],
                                as_index=False
                                )['Остаток руб'].mean()

df_clients = df.groupby(
                                by=['Артикул','КЛИЕНТ'],
                                as_index=False
                                )['Продажи руб'].sum().sort_values('Продажи руб', ascending=False)
display(df_clients.head(10))

df = pd.merge(left = df_stock, right= df_sales, on = 'Дата', how = 'outer')






df['медиана'] = df['Продажи руб'].median()
df['среднее'] = df['Продажи руб'].mean()
df['std'] = df['Продажи руб'].std()
df['страховой запас на день'] = df['Продажи руб'].mean() * ( 1+ df['std']/df['Продажи руб'].mean())
df['страховой запас'] = df['страховой запас на день']*5
display(df.head())
fig = px.line(df, x="Дата", y=['Продажи руб','Остаток руб', 'медиана','среднее','страховой запас на день','страховой запас'] ) 
fig.show()

fig = px.box(df, x='Продажи руб')
fig.show()

,Артикул,КЛИЕНТ,Продажи руб
0,349054/,ВАЙЛДБЕРРИЗ ООО,1970.82


,Дата,Остаток руб,Продажи руб,медиана,среднее,std,страховой запас на день,страховой запас
0,2022-07-24,0.0,1313.88,985.41,985.41,464.526729,1449.936729,7249.683644
1,2022-08-07,0.0,656.94,985.41,985.41,464.526729,1449.936729,7249.683644
